In [1]:
import os
# os.environ["VLLM_USE_V1"] = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

In [2]:
from vllm import LLM, SamplingParams
from vllm.inputs import TokensPrompt
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.generation.beam_search import *
from transformers.modeling_outputs import CausalLMOutputWithPast
from waterfall.WatermarkerBase import Watermarker

import types
import inspect
from copy import deepcopy
from timeit import timeit

In [3]:
base_model = 'meta-llama/Meta-Llama-3.1-8B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side='left'
watermarker = Watermarker(tokenizer)
logits_processor = watermarker.logits_processor

Cache size: 84538


In [4]:
max_tokens = max_new_tokens = 10

In [5]:
vllm = LLM(base_model,
            tokenizer=base_model, 
            gpu_memory_utilization=0.6, 
            max_model_len=50 + max_tokens,
            )

INFO 08-15 11:03:46 __init__.py:207] Automatically detected platform cuda.
INFO 08-15 11:04:07 config.py:549] This model supports multiple tasks: {'reward', 'embed', 'classify', 'generate', 'score'}. Defaulting to 'generate'.
INFO 08-15 11:04:07 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=60, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 08-15 11:04:28 model_runner.py:1115] Loading model weights took 14.9888 GB
INFO 08-15 11:04:29 worker.py:267] Memory profiling takes 0.44 seconds
INFO 08-15 11:04:29 worker.py:267] the current vLLM instance can use total_gpu_memory (79.10GiB) x gpu_memory_utilization (0.60) = 47.46GiB
INFO 08-15 11:04:29 worker.py:267] model weights take 14.99GiB; non_torch_memory takes 0.14GiB; PyTorch activation peak memory takes 0.24GiB; the rest of the memory reserved for KV Cache is 32.09GiB.
INFO 08-15 11:04:29 executor_base.py:111] # cuda blocks: 16429, # CPU blocks: 2048
INFO 08-15 11:04:29 executor_base.py:116] Maximum concurrency for 60 tokens per request: 4381.07x
INFO 08-15 11:04:32 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:23<00:00,  1.52it/s]

INFO 08-15 11:04:55 model_runner.py:1562] Graph capturing finished in 23 secs, took 0.32 GiB
INFO 08-15 11:04:55 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 26.46 seconds


In [6]:
try:
    del model
    torch.cuda.empty_cache()
except:
    pass
model = AutoModelForCausalLM.from_pretrained(base_model, torch_dtype=torch.bfloat16, device_map="auto")
model.eval()

# Save the original forward
model._original_forward = model.forward

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
test = AutoModelForCausalLM.from_pretrained(base_model, torch_dtype=torch.bfloat16, device_map="meta")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:

# def forward(
#     self,
#     input_ids: Optional[torch.LongTensor] = None,
#     attention_mask: Optional[torch.Tensor] = None,
#     position_ids: Optional[torch.LongTensor] = None,
#     past_key_values: Optional[Cache] = None,
#     inputs_embeds: Optional[torch.FloatTensor] = None,
#     labels: Optional[torch.LongTensor] = None,
#     use_cache: Optional[bool] = None,
#     cache_position: Optional[torch.LongTensor] = None,
#     logits_to_keep: Union[int, torch.Tensor] = 0,
#     **kwargs: Unpack[TransformersKwargs],
# ) -> CausalLMOutputWithPast:
#     r"""
#     Example:

#     ```python
#     >>> from transformers import AutoTokenizer, LlamaForCausalLM

#     >>> model = LlamaForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf")
#     >>> tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

#     >>> prompt = "Hey, are you conscious? Can you talk to me?"
#     >>> inputs = tokenizer(prompt, return_tensors="pt")

#     >>> # Generate
#     >>> generate_ids = model.generate(inputs.input_ids, max_length=30)
#     >>> tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
#     "Hey, are you conscious? Can you talk to me?\nI'm not conscious, but I can talk to you."
#     ```"""
#     outputs: BaseModelOutputWithPast = self.model(
#         input_ids=input_ids,
#         attention_mask=attention_mask,
#         position_ids=position_ids,
#         past_key_values=past_key_values,
#         inputs_embeds=inputs_embeds,
#         use_cache=use_cache,
#         cache_position=cache_position,
#         **kwargs,
#     )

#     hidden_states = outputs.last_hidden_state
#     # Only compute necessary logits, and do not upcast them to float if we are not computing the loss
#     slice_indices = slice(-logits_to_keep, None) if isinstance(logits_to_keep, int) else logits_to_keep
#     logits = self.lm_head(hidden_states[:, slice_indices, :])

#     loss = None
#     if labels is not None:
#         loss = self.loss_function(logits=logits, labels=labels, vocab_size=self.config.vocab_size, **kwargs)

#     return CausalLMOutputWithPast(
#         loss=loss,
#         logits=logits,
#         past_key_values=outputs.past_key_values,
#         hidden_states=outputs.hidden_states,
#         attentions=outputs.attentions,
#     )

In [8]:
prompt_fn = lambda content: tokenizer.apply_chat_template([
    {"role": "user", "content": content},
], tokenize = False, add_generation_prompt = True)

In [9]:
inputs = [prompt_fn("What is the meaning of life?"), prompt_fn("When was Obama born?")]
tokd_input = tokenizer(inputs, return_tensors="pt", add_special_tokens=False, padding=True).to(model.device)
num_beam_groups = 2
beams_per_group = 3

In [ ]:
tokd_input

{'input_ids': tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1627,  10263,    220,   2366,     19,    271, 128009, 128006,
            882, 128007,    271,   3923,    374,    279,   7438,    315,   2324,
             30, 128009, 128006,  78191, 128007,    271],
        [128009, 128009, 128000, 128006,   9125, 128007,    271,  38766,   1303,
          33025,   2696,     25,   6790,    220,   2366,     18,    198,  15724,
           2696,     25,    220,   1627,  10263,    220,   2366,     19,    271,
         128009, 128006,    882, 128007,    271,   4599,    574,   7250,   9405,
             30, 128009, 128006,  78191, 128007,    271]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [ ]:
out = model.generate(
                **tokd_input,
                max_new_tokens=1,
                logits_processor=[logits_processor],
                pad_token_id=tokenizer.eos_token_id,
                do_sample = False, temperature=None, top_p=None,
                num_beams = num_beam_groups * beams_per_group,
                num_beam_groups = num_beam_groups,
                num_return_sequences = num_beam_groups * beams_per_group,
                diversity_penalty = 0.5, use_cache = False
                )

Group Beam Search is scheduled to be moved to a `custom_generate` repository in v4.55.0. To prevent loss of backward compatibility, add `trust_remote_code=True` to your `generate` call.


In [10]:
N = test.lm_head.out_features

beam_search_params = SamplingParams(logprobs=20,
                                    max_tokens=1,
                                    temperature=1.0,
                                    detokenize=False)

In [11]:
input_ids = tokd_input["input_ids"]
prompts_batch = [TokensPrompt(prompt_token_ids=i) for i in input_ids]
output = vllm.generate(prompts_batch,
                                   sampling_params=beam_search_params,
                                   use_tqdm=False)
logits = []
size = (N,)
for o in output:
    o = o.outputs[0].logprobs[0]
    tensor = torch.full(size, float('-inf'))

    # Set values from the dictionary
    if o:  # only if dictionary is not empty
        indices = torch.tensor(list(o.keys()))
        values = torch.tensor([i.logprob for i in o.values()])
        tensor[indices] = values

    logits.append(tensor)
logits = torch.stack(logits).unsqueeze(1)
output_dict = CausalLMOutputWithPast(
        loss=None,
        logits=logits,
        past_key_values=None,
        hidden_states=None,
        attentions=None,
    )

In [ ]:
# Define the replacement

N = test.lm_head.out_features

beam_search_params = SamplingParams(logprobs=20,
                                    max_tokens=1,
                                    temperature=1.0,
                                    detokenize=False)

def new_forward(self, prompts_batch, input_ids, *args, **kwargs):
    prompts_batch = deepcopy(prompts_batch)
    for i, j in zip(input_ids, prompts_batch):
        j['prompt_token_ids'] += i[1:].tolist()
    output = vllm.generate(prompts_batch,
                                    sampling_params=beam_search_params,
                                    use_tqdm=False)
    logits = torch.full((len(output), N), float('-inf'))
    for i, o in enumerate(output):
        o = o.outputs[0].logprobs[0]
        indices = torch.tensor(list(o.keys()))
        values = torch.tensor([i.logprob for i in o.values()])
        logits[i, indices] = values

    logits = logits.unsqueeze(1)
    logits = logits.to(input_ids.device)
    # print(logits[:, -1].topk(5, dim=-1))
    # print(logits.shape)
    output_dict = CausalLMOutputWithPast(
            loss=None,
            logits=logits,
            past_key_values=None,
            hidden_states=None,
            attentions=None,
        )
    return output_dict

# --- Usage ---
print("Using patched forward:")
print("Patched forward signature:", set(inspect.signature(model.forward).parameters))
token_prompts = [TokensPrompt(prompt_token_ids=tokenizer.encode(i, add_special_tokens=False)) for i in inputs]
token_prompts = [j for i in token_prompts for j in [deepcopy(i) for _ in range(num_beam_groups * beams_per_group)]]
test.forward = types.MethodType(new_forward, test)
out = test.generate(
                input_ids = torch.full((len(inputs), 1), tokenizer.bos_token_id, dtype=torch.long, device=model.device),
                # **tokd_input,
                max_new_tokens=5,
                # logits_processor=[logits_processor],
                pad_token_id=tokenizer.eos_token_id,
                do_sample = False, temperature=None, top_p=None,
                num_beams = num_beam_groups * beams_per_group,
                num_beam_groups = num_beam_groups,
                num_return_sequences = num_beam_groups * beams_per_group,
                diversity_penalty = 0.5, use_cache = False,
                prompts_batch = token_prompts,
                output_scores=True, return_dict_in_generate=True
                )
print(out.sequences_scores)
print(tokenizer.batch_decode(out.sequences[:,1:]))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Using patched forward:
Patched forward signature: {'position_ids', 'past_key_values', 'kwargs', 'cache_position', 'attention_mask', 'use_cache', 'logits_to_keep', 'labels', 'inputs_embeds', 'input_ids'}
791 {791: Logprob(logprob=-0.020645949989557266, rank=1, decoded_token=None), 4054: Logprob(logprob=-4.645646095275879, rank=2, decoded_token=None), 32: Logprob(logprob=-5.020646095275879, rank=3, decoded_token=None), 3923: Logprob(logprob=-6.145646095275879, rank=4, decoded_token=None), 30690: Logprob(logprob=-6.770646095275879, rank=5, decoded_token=None), 20397: Logprob(logprob=-7.520646095275879, rank=6, decoded_token=None), 3947: Logprob(logprob=-9.395646095275879, rank=7, decoded_token=None), 8142: Logprob(logprob=-9.833146095275879, rank=8, decoded_token=None), 31140: Logprob(logprob=-9.895646095275879, rank=9, decoded_token=None), 16533: Logprob(logprob=-10.645646095275879, rank=10, decoded_token=None), 32576: Logprob(logprob=-10.833146095275879, rank=11, decoded_token=None), 20

/home/niuxy/maplecg_nfs/miniconda3/envs/vllm_waterfall/lib/python3.12/site-packages/transformers/generation/utils.py:2506: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cuda, whereas the model is on meta. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('meta') before running `.generate()`.
  warnings.warn(


315 {315: Logprob(logprob=-0.006171337328851223, rank=1, decoded_token=None), 330: Logprob(logprob=-5.131171226501465, rank=2, decoded_token=None), 364: Logprob(logprob=-8.631171226501465, rank=3, decoded_token=None), 430: Logprob(logprob=-10.131171226501465, rank=4, decoded_token=None), 922: Logprob(logprob=-11.506171226501465, rank=5, decoded_token=None), 11: Logprob(logprob=-12.131171226501465, rank=6, decoded_token=None), 499: Logprob(logprob=-12.631171226501465, rank=7, decoded_token=None), 1054: Logprob(logprob=-13.381171226501465, rank=8, decoded_token=None), 5196: Logprob(logprob=-13.881171226501465, rank=9, decoded_token=None), 374: Logprob(logprob=-14.568671226501465, rank=10, decoded_token=None), 389: Logprob(logprob=-14.943671226501465, rank=11, decoded_token=None), 14932: Logprob(logprob=-15.068671226501465, rank=12, decoded_token=None), 706: Logprob(logprob=-15.256171226501465, rank=13, decoded_token=None), 311: Logprob(logprob=-15.318671226501465, rank=14, decoded_token=

In [ ]:
def print_forward(self, *args, **kwargs):
    out = self.__class__.forward(self, *args, **kwargs)
    out.logits = out.logits.float()
    # logits = out.logits
    # print(logits[:, -1].float().log_softmax(dim=-1).topk(5, dim=-1))
    return out

# Swap back to the original
# model.forward = model._original_forward
model.forward = types.MethodType(print_forward, model)
print("\nUsing original forward:")
print("Restored forward signature:", set(inspect.signature(model.forward).parameters))
out = model.generate(
                **tokd_input,
                max_new_tokens=5,
                # logits_processor=[logits_processor],
                pad_token_id=tokenizer.eos_token_id,
                do_sample = False, temperature=None, top_p=None,
                num_beams = num_beam_groups * beams_per_group,
                num_beam_groups = num_beam_groups,
                num_return_sequences = num_beam_groups * beams_per_group,
                diversity_penalty = 0.5, use_cache = False,
                output_scores=True, return_dict_in_generate=True
                )
print(out.sequences_scores)
print(tokenizer.batch_decode(out.sequences[:,tokd_input["input_ids"].shape[1]:]))


Using original forward:
Restored forward signature: {'args', 'kwargs'}
tensor([-0.0608, -0.3091, -0.9608, -1.0058, -1.0091, -1.0627, -0.0466, -0.3464,
        -0.5466, -0.8464, -0.8951, -1.3951], device='cuda:0')
['The question of the meaning', 'The meaning of life is', 'The question of the meaning', 'The concept of the meaning', 'The meaning of life is', 'One of the most profound', 'Barack Obama, the', 'Barack Obama was born', 'Barack Obama, the', 'Barack Obama was born', 'Former US President Barack Obama', 'Former US President Barack Obama']


In [ ]:
inputs_ = inputs[:1]
tokd_input_ = tokenizer(inputs_, return_tensors="pt", add_special_tokens=False, padding=True).to(model.device)

In [ ]:
# token_prompts = [TokensPrompt(prompt_token_ids=tokenizer.encode(i, add_special_tokens=False)) for i in inputs_ * 100]
# token_prompts = [j for i in token_prompts for j in [deepcopy(i) for _ in range(num_beam_groups * beams_per_group)]]
# model.forward = types.MethodType(new_forward, model)
# timeit(lambda: model.generate(
#                 input_ids = torch.full((len(inputs_), 1), tokenizer.bos_token_id, dtype=torch.long, device=model.device),
#                 # **tokd_input,
#                 max_new_tokens=10,
#                 # logits_processor=[logits_processor],
#                 pad_token_id=tokenizer.eos_token_id,
#                 do_sample = False, temperature=None, top_p=None,
#                 num_beams = num_beam_groups * beams_per_group,
#                 num_beam_groups = num_beam_groups,
#                 num_return_sequences = num_beam_groups * beams_per_group,
#                 diversity_penalty = 0.5, use_cache = False,

#                 prompts_batch = token_prompts
#                 ), number=1)

In [ ]:

# model.forward = model._original_forward
# timeit(lambda: model.generate(
#                 **tokd_input_,
#                 max_new_tokens=10,
#                 # logits_processor=[logits_processor],
#                 pad_token_id=tokenizer.eos_token_id,
#                 do_sample = False, temperature=None, top_p=None,
#                 num_beams = num_beam_groups * beams_per_group,
#                 num_beam_groups = num_beam_groups,
#                 num_return_sequences = num_beam_groups * beams_per_group,
#                 diversity_penalty = 0.5, use_cache = False
#                 ), number=100)